自动求导里面其实最主要的就是，搞清楚矩阵（向量）求导时候他们的维度变化

这其实和前面的重点是完全一样的

自动求导其实是计算函数在某一个特定值上的导数

1. 我们人手动计算的时候，通常会用符号求导（就是那些求导法则），这样得到的是显式解
2. 另外的，我们也可以通过数值求导，不求显式解，直接计算函数在某个位置上的导数值（当然这是一个逼近值）
3. 数值求导看起来和自动求导有点像，但是有个问题，我们需要知道f(x)的显式解才能进行数值求导（因为要算f(x+h)），但是这在深度学习里面挺麻烦的

自动求导会涉及到一个计算图的概念，按照我的理解，所谓计算图其实就是从最基础的自变量开始，通过构造中间变量并且互相组合来形成最后的因变量（也就是整个函数）

计算图中，每一个圈其实就是一个操作，这个操作可以是c=a+b这种生成新的变量，也可以是单纯的引入变量，也就是引入x啥的

比如数学上，我们常常会定义c=a+b这种式子，这玩意实际上就是个计算图

有了计算图之后，我们就可以让计算机来理解链式法则了（是的，我感觉计算图的最大作用就是计算机能够理解链式法则）

借由链式法则，可以有以下两种自动求导的计算方法：
1. 正向累积（传递）：
所谓正向，其实就是按照计算图的方向来进行的，也就是在链式法则中，先计算最基础的关于自变量x的导数，再算各种中间变量的导数，最后乘起来
2. 反向累积（传递）
反向就是相反的，与计算图的构造方向反着来，先从因变量计算最后一个中间变量，然后一直算一直算，最后算到中间变量关于x的导数，全部相乘
3. 其实可以发现，我们人类手动计算，是按着反向传递的方法来算的

为啥现在，神经网络里面都是用反向传播算法去自动求导呢？这个结合李沐老师的视频课件比较好理解
1. 计算复杂度正向是O(n)
2. 内存复杂度上来说，正向是O(1)，反向还是O(n)，因为正向在不断构造计算图的过程中就已经求导了，而反向是要储存中间变量，不能像正向一样删掉
3. 上面内存复杂度的差别也导致了，为啥现在深度学习训练要那么夸张的显存
4. 在计算复杂度上，反向我认为有很大的优势，通过计算图可以发现，反向在返回去计算的时候，可以完全不管一些无关的变量，因为和我们要求的导数没关系，肯定是0的，这样计算的开销就比正向那种所有操作子都得算的O(n)要好很多
5. 而且对神经网络来说，我们常常要对每一层都计算梯度，那正向每算一次都是O(n)的计算消耗，这样对于一些深一点的神经网络来说计算消耗太恐怖了
5. 所以从计算的耗费资源上来看，反向才被广泛应用到现在的深度学习中，不过这也导致了现在需要很大的显存才能训练

下面是一些自动求导的实现

In [122]:
import torch

x = torch.arange(4.0, requires_grad=True) # 这里是告诉torch，我要有一个地方来存我这个变量的梯度（导数），不然没地方存，肯定就算不了

In [123]:
y = 2 * torch.dot(x, x)
y # 这里可以发现，y里面跟了一个grad_fn，这其实就是我们上面构造x的时候说了我们需要梯度，所以torch帮我们隐式的生成了一个梯度函数，tf/mxnet这些可以显式的定义

tensor(28., grad_fn=<MulBackward0>)

下面我们就是调用了一个反向传播函数，来计算y关于x的函数

In [124]:
y.backward()
x.grad == 4 * x

tensor([True, True, True, True])

需要注意的是，默认情况下torch每次计算梯度都会把它累积起来，所以需要我们人为的清除一下，不然算出来的肯定是错的

torch里面下划线结尾的函数意思就是，对内容进行更改

In [125]:
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

在上面，我们讨论的都是一个标量对一个向量去求梯度，接下来，我们看看向量对向量求梯度是啥样子的

不过，在深度学习中，基本上都是标量对向量求梯度

In [126]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
# y.backward(torch.tensor([1.0, 1.0, 1.0, 1.0]))
x.grad

tensor([0., 2., 4., 6.])

上面那个，“该参数指定微分函数关于self的梯度”，看起来很抽象，实际上呢，是我们可以指定y对自己的梯度是啥

比如上面那个例子，我们反向传播可以分解为，y=a/a=x2

那我们求y对a的梯度，正常来说肯定会是1对吧，但是我们可以人为的设置成别的，比如1234

这样，我们就会得到与之前完全不一样的结果

但是一般来说，我们求一个向量的梯度，都会直接对它进行求和操作得到一个标量再去求梯度（和上面的例子一样）

有时候我们也想把某些变量移到计算图之外，这样其实是起到了一个固定参数的作用，从而避免这个参数被更新（也就是变成像常数一样了）

In [127]:
x.grad.zero_()
y = x * x
u = y.detach() # 这里detach就是把u变成了一个标量
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [128]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

有时候我们定义的一个计算图，可能是通过复杂的控制流，比如if/else啥的来实现的

torch非常强大，针对这种复杂的关系，也能隐式的自动生成计算图，不过比tf或者mxnet这种显式的构造计算上要慢一点

In [129]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)

## 小结

* 深度学习框架可以自动计算导数：我们首先将梯度附加到想要对其计算偏导数的变量上，然后记录目标值的计算，执行它的反向传播函数，并访问得到的梯度。

上面的注意：
1. 梯度的需求一定是附加到我们想要计算梯度的变量上，比如x
2. 我们一定要构造好计算图，其实也就是目标值（或者目标函数）的计算过程，这样计算机才能隐式的构造计算图
3. 上面的做好之后，就可以对目标值/函数进行反向传播计算了